In [2]:
import pandas as pd
import numpy as np
from thefuzz import process

In [3]:
#df_listino_gen = pd.read_csv('data_generated/listino_completo.csv')
#df_listino = pd.read_excel('data_clean/art_costi.xlsx')
## Find codes present in df_listino_gen but not in df_listino
#codes_listino_gen = df_listino_gen['Codice_articolo'].to_list()
#codes_listino = df_listino['Codice_articolo'].to_list()
#
#codes_to_insert = list(set(codes_listino_gen) - set(codes_listino))
#
## Filter rows from df_listino_gen for those codes
#df_to_insert = df_listino_gen[df_listino_gen['Codice_articolo'].isin(codes_to_insert)]
#
## Concatenate DataFrames to insert new products
#df_listino = pd.concat([df_listino, df_to_insert], ignore_index=True)
#df_listino

In [4]:
df_listino = pd.read_csv('data_generated/listino_completo.csv')
#df_listino = pd.read_excel('data_clean/art_costi.xlsx')
df_ddt = pd.read_excel('data_clean/ddt_righe.xlsx')

In [5]:
IDs_listino_univoci = df_listino['Codice_articolo'].unique().tolist()
n_IDs_listino = len(IDs_listino_univoci)
IDs_ddt = df_ddt['Codice_articolo'].tolist()
IDs_ddt_univoci = df_ddt['Codice_articolo'].unique().tolist()
n_IDs_ddt = df_ddt['Codice_articolo'].nunique()

In [6]:
IDs_ddt_not_in_listino = [articolo for articolo in IDs_ddt if articolo not in IDs_listino_univoci]
IDs_ddt_not_in_listino_univoci = list(set(IDs_ddt_not_in_listino))
n_IDs_ddt_not_in_listino = len(IDs_ddt_not_in_listino)
n_IDs_ddt_not_in_listino_univoci = len(IDs_ddt_not_in_listino_univoci)

IDs_listino_not_in_ddt_univoci = [articolo for articolo in IDs_listino_univoci if articolo not in IDs_ddt_univoci]
n_IDs_listino_not_in_ddt = len(IDs_listino_not_in_ddt_univoci)

print('Numero articoli nel listino: ', n_IDs_listino)
print('Numero articoli nei DDT: ', n_IDs_ddt)
print('Numero articoli nei DDT non presenti nel listino: ', n_IDs_ddt_not_in_listino)
print('Numero articoli nei DDT non presenti nel listino (univoci): ', n_IDs_ddt_not_in_listino_univoci)
print('Numero articoli nel listino non presenti nei DDT (univoci): ', n_IDs_listino_not_in_ddt)
print('Percentuale articoli nei DDT non presenti nel listino (univoci): ', n_IDs_ddt_not_in_listino_univoci/n_IDs_ddt)
print('Percentuale articoli nel listino non presenti nei DDT (univoci): ', n_IDs_listino_not_in_ddt/n_IDs_listino)

Numero articoli nel listino:  988
Numero articoli nei DDT:  1168
Numero articoli nei DDT non presenti nel listino:  12912
Numero articoli nei DDT non presenti nel listino (univoci):  480
Numero articoli nel listino non presenti nei DDT (univoci):  300
Percentuale articoli nei DDT non presenti nel listino (univoci):  0.410958904109589
Percentuale articoli nel listino non presenti nei DDT (univoci):  0.30364372469635625


In [7]:
dict_result = {'ID DDT':[], 'ID più simile nel listino':[], 'score':[]}
for i in range(0, n_IDs_ddt_not_in_listino_univoci):
    dict_result['ID DDT'].append(IDs_ddt_not_in_listino_univoci[i])
    dict_result['ID più simile nel listino'].append(process.extract(IDs_ddt_not_in_listino_univoci[i], IDs_listino_univoci, limit=1)[0][0])
    dict_result['score'].append(process.extract(IDs_ddt_not_in_listino_univoci[i], IDs_listino_univoci, limit=1)[0][1])
df_ddt_simili_listino = pd.DataFrame(dict_result)
df_ddt_simili_listino

,ID DDT,ID più simile nel listino,score
0,017.TD.HOP.001.58,017.PM419,86
1,091.NW2531AN,091.NW2531,91
2,017.TD.VIT.014.5,017.PM419,86
3,038.STC58.2D170.01,017.PM105.01,86
4,091.NW2547AN,091.NW2547,91
...,...,...,...
475,017.TD.COM.001.1230,017.PM419,86
476,017.IMB22,017.GUI.MED.01,86
477,028.266VFOR,028.220,86
478,028.PRES2,028.220,62


In [8]:
dict_result = {'ID listino':[], 'ID più simile nei DDT':[], 'score':[]}
for i in range(0, n_IDs_listino_not_in_ddt):
    dict_result['ID listino'].append(IDs_listino_not_in_ddt_univoci[i])
    dict_result['ID più simile nei DDT'].append(process.extract(IDs_listino_not_in_ddt_univoci[i], IDs_ddt_univoci, limit=1)[0][0])
    dict_result['score'].append(process.extract(IDs_listino_not_in_ddt_univoci[i], IDs_ddt_univoci, limit=1)[0][1])
df_listino_simili_ddt = pd.DataFrame(dict_result)
df_listino_simili_ddt

,ID listino,ID più simile nei DDT,score
0,017.PM420,017.PM420_AL,95
1,017.PM423,017.PM438,89
2,017.PM429,017.PM425,89
3,017.PM484,017.PM438,89
4,017.PM542,017.PM152,89
...,...,...,...
295,052.MP.POL35.37,052.MP.PES35.37,87
296,052.MP.POL35.43,052.MP.PES35.43,87
297,052.MP.POL35.54,052.MP.PES35.54,87
298,052.MP.POL35.60,052.MP.PES35.60,87


In [9]:
#save both dataframes to csv
df_ddt_simili_listino.to_csv('data_generated/ddt_simili_listino.csv', index=False)
df_listino_simili_ddt.to_csv('data_generated/listino_simili_ddt.csv', index=False)

In [10]:
# remove ids not present in the listino from the ddt
df_ddt = df_ddt[df_ddt['Codice_articolo'].isin(IDs_listino_univoci)]
df_ddt.to_csv('data_demo/ddt_righe.csv', index=False)
# remove ids not present in the ddt from the listino
df_listino = df_listino[df_listino['Codice_articolo'].isin(IDs_ddt_univoci)]
df_listino.to_csv('data_demo/art_costi.csv', index=False)